In [35]:
!pip install z3-solver

In [36]:
!pip install nashpy

In [37]:
from itertools import product
import itertools
import numpy as np
from z3 import *
import nashpy

In [38]:
def is_best_response(player, action, full_profile, players, action_sets, payoff_matrices):
    current_profile = list(full_profile)
    current_profile[player] = action
    current_profile = tuple(current_profile)

    current_payoff = payoff_matrices[player][current_profile]

    for alt_action in action_sets[player]:
        if alt_action == action:
            continue
        alt_profile = list(full_profile)
        alt_profile[player] = alt_action
        alt_profile = tuple(alt_profile)
        if payoff_matrices[player][alt_profile] > current_payoff:
            return False
    return True

def find_pure_nash_equilibria(players, action_sets, payoff_matrices):
    equilibria = []

    all_profiles = product(*[action_sets[i] for i in players])

    for profile in all_profiles:
        if all(is_best_response(i, profile[i], profile, players, action_sets, payoff_matrices) for i in players):
            equilibria.append(profile)

    return equilibria


In [39]:
def find_mixed_strategy_nash(players, action_sets, payoff_matrices):
    s = Solver()

    strategy_vars = {
        i: [Real(f"p_{i}_{a}") for a in range(len(action_sets[i]))]
        for i in players
    }

    for i in players:
        for p in strategy_vars[i]:
            s.add(p >= 0, p <= 1)
        s.add(Sum(strategy_vars[i]) == 1)

    index_profiles = list(product(*[range(len(action_sets[i])) for i in players]))

    for i in players:
        for a_i in range(len(action_sets[i])):
            expected_current = RealVal(0)
            expected_actual = RealVal(0)

            for profile in index_profiles:
                prob_expr = RealVal(1)
                for j in players:
                    prob_expr *= strategy_vars[j][profile[j]]

                profile_named_actual = tuple(action_sets[j][profile[j]] for j in players)

                profile_current = list(profile)
                profile_current[i] = a_i
                profile_named_current = tuple(action_sets[j][profile_current[j]] for j in players)

                expected_current += prob_expr * payoff_matrices[i][profile_named_current]
                expected_actual += prob_expr * payoff_matrices[i][profile_named_actual]

            s.add(expected_current >= expected_actual)

    if s.check() == sat:
        m = s.model()
        print("Mixed Strategy Nash Equilibrium:")
        for i in players:
            print(f"Player {i}:")
            for idx, var in enumerate(strategy_vars[i]):
                val = m.evaluate(var, model_completion=True)
                prob = float(val.numerator_as_long()) / float(val.denominator_as_long())
                action = action_sets[i][idx]
                print(f"  {action}: {prob:.3f}")
    else:
        print("No Nash Equilibrium found.")

In [40]:
# # example 1
# players = [0, 1, 2]

# action_sets = {
#     0: [0, 1, 2],
#     1: [0, 1, 2],
#     2: [0, 1, 2]
# }

# payoff_matrices = {
#     0: {
#         (0, 0, 0): 3, (0, 0, 1): 3, (0, 0, 2): 3,
#         (0, 1, 0): 3, (0, 1, 1): 3, (0, 1, 2): 3,
#         (0, 2, 0): 3, (0, 2, 1): 3, (0, 2, 2): 3,
#         (1, 0, 0): 1, (1, 0, 1): 1, (1, 0, 2): 1,
#         (1, 1, 0): 1, (1, 1, 1): 1, (1, 1, 2): 1,
#         (1, 2, 0): 1, (1, 2, 1): 1, (1, 2, 2): 1,
#         (2, 0, 0): 0, (2, 0, 1): 0, (2, 0, 2): 0,
#         (2, 1, 0): 0, (2, 1, 1): 0, (2, 1, 2): 0,
#         (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0,
#     },
#     1: {
#         (0, 0, 0): 2, (0, 0, 1): 2, (0, 0, 2): 2,
#         (0, 1, 0): 1, (0, 1, 1): 1, (0, 1, 2): 1,
#         (0, 2, 0): 0, (0, 2, 1): 0, (0, 2, 2): 0,
#         (1, 0, 0): 2, (1, 0, 1): 2, (1, 0, 2): 2,
#         (1, 1, 0): 1, (1, 1, 1): 1, (1, 1, 2): 1,
#         (1, 2, 0): 0, (1, 2, 1): 0, (1, 2, 2): 0,
#         (2, 0, 0): 2, (2, 0, 1): 2, (2, 0, 2): 2,
#         (2, 1, 0): 1, (2, 1, 1): 1, (2, 1, 2): 1,
#         (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0,
#     },
#     2: {
#         (0, 0, 0): 1, (0, 0, 1): 0, (0, 0, 2): 2,
#         (0, 1, 0): 1, (0, 1, 1): 0, (0, 1, 2): 2,
#         (0, 2, 0): 1, (0, 2, 1): 0, (0, 2, 2): 2,
#         (1, 0, 0): 1, (1, 0, 1): 0, (1, 0, 2): 2,
#         (1, 1, 0): 1, (1, 1, 1): 0, (1, 1, 2): 2,
#         (1, 2, 0): 1, (1, 2, 1): 0, (1, 2, 2): 2,
#         (2, 0, 0): 1, (2, 0, 1): 0, (2, 0, 2): 2,
#         (2, 1, 0): 1, (2, 1, 1): 0, (2, 1, 2): 2,
#         (2, 2, 0): 1, (2, 2, 1): 0, (2, 2, 2): 2,
#     }
# }

In [47]:
#example 6.2 from book
players = [0, 1]

action_sets = {
    0: ['NC', 'C'],
    1: ['NC', 'C']
}

payoff_matrices = {
    0: {
        ('NC', 'NC'): -2,
        ('NC', 'C'): -10,
        ('C', 'NC'): -1,
        ('C', 'C'): -5,
    },
    1: {
        ('NC', 'NC'): -2,
        ('NC', 'C'): -1,
        ('C', 'NC'): -10,
        ('C', 'C'): -5,
    }
}

In [50]:
# # example 6.6 from book
# players = [0, 1]

# action_sets = {
#     0: ['A', 'B'],  # Player 1's actions
#     1: ['A', 'B']   # Player 2's actions
# }

# payoff_matrices = {
#     0: {  # Player 1's payoffs
#         ('A', 'A'): -1,
#         ('A', 'B'): -0.5,
#         ('B', 'A'): -1,
#         ('B', 'B'): -1
#     },
#     1: {  # Player 2's payoffs
#         ('A', 'A'): -1,
#         ('A', 'B'): -1,
#         ('B', 'A'): -0.5,
#         ('B', 'B'): -1
#     }
# }


In [52]:
# #example 6.9 from book
# players = [0, 1]

# action_sets = {
#     0: ['Rock', 'Paper', 'Scissors'],
#     1: ['Rock', 'Paper', 'Scissors']
# }

# payoff_matrices = {
#     0: {  # Player 1's payoffs
#         ('Rock', 'Rock'): 0,     ('Rock', 'Paper'): -1,  ('Rock', 'Scissors'): 1,
#         ('Paper', 'Rock'): 1,    ('Paper', 'Paper'): 0,  ('Paper', 'Scissors'): -1,
#         ('Scissors', 'Rock'): -1,('Scissors', 'Paper'): 1, ('Scissors', 'Scissors'): 0
#     },
#     1: {  # Player 2's payoffs
#         ('Rock', 'Rock'): 0,     ('Rock', 'Paper'): 1,   ('Rock', 'Scissors'): -1,
#         ('Paper', 'Rock'): -1,   ('Paper', 'Paper'): 0,  ('Paper', 'Scissors'): 1,
#         ('Scissors', 'Rock'): 1, ('Scissors', 'Paper'): -1, ('Scissors', 'Scissors'): 0
#     }
# }


In [44]:
# #video example

# players = [0, 1, 2]  # 0: Gus, 1: Yelnic, 2: Tolbert

# action_sets = {
#     0: ['Goes to bar', 'Stays home'],  # Gus
#     1: ['go to bar', 'stays home'],    # Yelnic
#     2: ['Goes to bar', 'Stays home']   # Tolbert
# }

# payoff_matrices = {
#     0: {  # Gus
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 2,
#         ('Goes to bar', 'go to bar', 'Stays home'): 2,
#         ('Goes to bar', 'stays home', 'Stays home'): 0,
#         ('Stays home', 'go to bar', 'Goes to bar'): 1,
#         ('Stays home', 'stays home', 'Goes to bar'): 1,
#         ('Stays home', 'go to bar', 'Stays home'): 1,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     },
#     1: {  # Yelnic
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 1,
#         ('Goes to bar', 'go to bar', 'Stays home'): 2,
#         ('Goes to bar', 'stays home', 'Stays home'): 1,
#         ('Stays home', 'go to bar', 'Goes to bar'): 2,
#         ('Stays home', 'stays home', 'Goes to bar'): 1,
#         ('Stays home', 'go to bar', 'Stays home'): 0,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     },
#     2: {  # Tolbert
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 2,
#         ('Goes to bar', 'go to bar', 'Stays home'): 1,
#         ('Goes to bar', 'stays home', 'Stays home'): 1,
#         ('Stays home', 'go to bar', 'Goes to bar'): 2,
#         ('Stays home', 'stays home', 'Goes to bar'): 0,
#         ('Stays home', 'go to bar', 'Stays home'): 1,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     }
# }


In [53]:
def get_input():
    num_players = int(input("Enter number of players: "))
    players = [i for i in range(num_players)]
    action_sets = {}
    for p in range(num_players):
        actions = input(f"Enter space-separated action for player {p} : ").split()
        action_sets[p] = [a for a in actions]

    all_profiles = list(product(*[action_sets[i] for i in range(num_players)]))
    payoff_matrices = {p: {} for p in range(num_players)}

    print("The index represents players i choice eg (C, NC) is player 0, player 1")
    for player in range(num_players):
        print(f"For Player {player}")
        for profile in all_profiles:
            input_utility = int(input(f"Profile {profile}: "))
            # print(input_utility)
            payoff_matrices[player][profile] = input_utility

    return players, action_sets, payoff_matrices

# players, action_sets, payoff_matrices = get_input()
# print(players, action_sets, payoff_matrices)

In [54]:
# part a
nash_equilibria = find_pure_nash_equilibria(players, action_sets, payoff_matrices)
if nash_equilibria:
    print("Pure Strategy Nash Equilibria:")
    for eq in nash_equilibria:
        print(eq)
else:
    print("No Pure Strategy Nash Equilibrium exists.")

# PART B
find_mixed_strategy_nash(players, action_sets, payoff_matrices)

No Pure Strategy Nash Equilibrium exists.
Mixed Strategy Nash Equilibrium:
Player 0:
  Rock: 0.333
  Paper: 0.333
  Scissors: 0.333
Player 1:
  Rock: 0.333
  Paper: 0.333
  Scissors: 0.333
